In [84]:
import sys
import os

import numpy as np
from matplotlib import pyplot as plt
from sqlalchemy.dialects.mssql.information_schema import columns

sys.path.append(os.path.abspath(".."))

import importlib
import pandas as pd

import volpy_func_lib as vp
import load_clean_lib
import table_lib
import vol_strat_lib as vs
import volpy_func_ticker_lib as vtp

importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)
importlib.reload(vs)
importlib.reload(vtp)

os.makedirs("figures/Analysis", exist_ok=True)
os.makedirs("figures/Analysis/Profitability", exist_ok=True)
os.makedirs("figures/Analysis/Factor models", exist_ok=True)
os.makedirs("figures/Analysis/Implementability", exist_ok=True)
os.makedirs("figures/Analysis/Period", exist_ok=True)

In [85]:
df_bid_ALL = vtp.concat_ticker_datasets(vp.All_tickers, df_name="sum1", folder_version = "_bid")
df_ask_ALL = vtp.concat_ticker_datasets(vp.All_tickers, df_name="sum1", folder_version = "_ask")

# df_mid_ALL = vtp.concat_ticker_datasets(vp.All_tickers, df_name="sum1", folder_version = "_mid")
# df_om_ALL = vtp.concat_ticker_datasets(vp.All_tickers, df_name="sum1", folder_version = "_om")

Skipped 2/237 tickers: ['RUT', 'INDU']
Skipped 2/237 tickers: ['RUT', 'INDU']


D:\Professionelt\Code\volpy\volpy_func_ticker_lib.py:1115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_merged = pd.concat(frames, ignore_index=True)


In [91]:
importlib.reload(vtp)
importlib.reload(table_lib)

ticker_list_list = [vp.VIX_tickers, vp.Cross_AM_tickers, vp.OEX_tickers, vp.DJX_tickers, vp.Liquid_tickers]
name_list = ["VIX", "Cross-AM", "OEX", "DJX", "Liquid"]

# ticker_list_list = [vp.Liquid_tickers]
# name_list = ["Liquid"]

for ticker_list, name in zip(ticker_list_list, name_list):
    
    df_bid = df_bid_ALL[df_bid_ALL["ticker"].isin(ticker_list)].copy()
    df_ask = df_ask_ALL[df_ask_ALL["ticker"].isin(ticker_list)].copy()

    t2_bid = table_lib.CarrWu2009_table_2(df_bid, name)
    t2_ask = table_lib.CarrWu2009_table_2(df_ask, name)
    
    t3_bid = table_lib.CarrWu2009_table_3(df_bid, name)
    t3_ask = table_lib.CarrWu2009_table_3(df_ask, name)
    
    df_bid["r_stock"] = 0
    df_bid["ln RV/SW 30"] = np.log(df_bid["RV"]) - np.log(df_bid["SW_0_30"])
    returns_df_bid = vp.make_df_strats(df_bid, sgy_common = "r_D_30_", sgy_names = [], plot = False, factors=["SPX", "Mkt", "SMB", "HML", "UMD", "BAB"], ticker_list= ticker_list, extra_columns=["#K", "RV", "r_30_SW_day .20", "RV-SW 30", "ln RV/SW 30"], sign=False, scale=False)
    tff5_bid = table_lib.save_ff5_table(returns_df_bid, name, max_lags = 21, y_var = "ln RV/SW 30")
    tff5_bid = tff5_bid.rename(columns={"Ticker": "ticker"})
    
    df_ask["r_stock"] = 0
    # df_ask["ln RV/SW 30"] = np.log(df_ask["RV"]) - np.log(df_ask["SW_0_30"])
    returns_df_ask = vp.make_df_strats(df_ask, sgy_common = "r_D_30_", sgy_names = [], plot = False, factors=["SPX", "Mkt", "SMB", "HML", "UMD", "BAB"], ticker_list= ticker_list, extra_columns=["#K", "RV", "r_30_SW_day .20", "RV-SW 30", "ln RV/SW 30"], sign=False, scale=False)
    tff5_ask = table_lib.save_ff5_table(returns_df_ask, name, max_lags = 21, y_var = "ln RV/SW 30")
    tff5_ask = tff5_ask.rename(columns={"Ticker": "ticker"})
    
    
    latex_1 = table_lib.make_bid_ask_table(t2_bid, t3_bid, t2_ask, t3_ask,
                                name = name,
                                caption="bid and ask results for SW, VRP and LVRP",
                                label="tab:SW, VRP and LVRP bid_ask"
                                )
    
    latex_2 = table_lib.make_bid_ask_table_ff5(t2_bid, t3_bid, t2_ask, t3_ask, tff5_bid, tff5_ask,
                                    name=name,
                                    caption="bid and ask results for SW, VRP, LVRP and alpha",
                                    label="tab:SW, VRP, LVRP and alpha bid_ask"
                                    )